In [8]:
import pandas as pd
import sqlite3
import math
import datetime 
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

In [9]:
conn = sqlite3.connect('database.db')

In [10]:
c = conn.cursor()

## DESAIN DATABASE DATAWAREHOUSE

![](desaindatabase.png)

In [11]:
c.executescript('drop table if exists Barang')
c.executescript('drop table if exists Pembelian')
c.executescript('drop table if exists Penjualan')
c.executescript('drop table if exists LabaRugi')
c.execute("CREATE TABLE IF NOT EXISTS 'Barang' ('id' INT, 'nama_barang' VARCHAR(255) NULL, PRIMARY KEY ('id'))")
c.execute("CREATE TABLE IF NOT EXISTS 'Pembelian' ('id' INT, 'qty_beli' INT NULL, 'harga_beli' FLOAT NULL, 'Barang_id' INT NOT NULL, PRIMARY KEY ('id'), FOREIGN KEY ('Barang_id') REFERENCES 'Barang' ('id') ON DELETE NO ACTION ON UPDATE NO ACTION)")
c.execute("CREATE TABLE IF NOT EXISTS 'Penjualan' ('id' INT, 'tanggal_jual' DATE NULL, 'qty_jual' INT NULL, 'harga_jual' FLOAT NULL, 'Pembelian_id' INT NOT NULL, PRIMARY KEY ('id'), FOREIGN KEY ('Pembelian_id') REFERENCES 'Pembelian' ('id') ON DELETE NO ACTION ON UPDATE NO ACTION)")
c.execute("CREATE TABLE IF NOT EXISTS 'LabaRugi' ('id' INT, 'total_pembelian' FLOAT NULL, 'total_penjualan' FLOAT NULL, 'Pembelian_id' INT NOT NULL, 'Penjualan_id' INT NOT NULL, PRIMARY KEY ('id'), FOREIGN KEY ('Pembelian_id') REFERENCES 'Pembelian' ('id') ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ('Penjualan_id') REFERENCES 'Penjualan' ('id') ON DELETE NO ACTION ON UPDATE NO ACTION)")

In [12]:
df = pd.read_csv("DATA SET LABA RUGI.csv")
df

,Nota,Barang,Harga Pembelian,QTY Pembelian,Harga Jual,Tanggal Penjualan,QTY penjualan,Pembelian Total,Penjualan Total,Laba / Rugi
0,1,JAKET,245000,100,250000,01/01/2021,50,24500000,12500000,500000
1,2,KEMEJA FLANEL BENHILL,287000,100,300000,01/01/2021,90,28700000,27000000,1300000
2,3,CELANA CHINOS EMBA,260000,100,267000,01/01/2021,95,26000000,25365000,700000
3,4,KAOS 3SECOND,145000,100,150000,01/01/2021,80,14500000,12000000,500000
4,5,CELANA PENDEK NEVADA KNIT BASIC,190000,100,199000,01/01/2021,78,19000000,15522000,900000
...,...,...,...,...,...,...,...,...,...,...
58,59,ST.YVES LONG SLEEVE DRESS,118000,90,124900,01/01/2021,56,11241000,6994400,621000
59,60,ST.YVES PLAIN PANTS RAYON,65000,90,70000,01/01/2021,34,6300000,2380000,450000
60,61,ST.YVES DRESS PRINT LEAF,140000,90,149900,01/01/2021,56,13491000,8394400,891000
61,62,TULIP SIMPLE SOFT GIRLS PANTY,59000,90,64000,01/01/2021,77,5760000,4928000,450000


In [13]:
barang = []
pembelian = []
for i,j in df.iterrows():
    if j[1] not in barang:
        barang.append(j[1])
        
    k = -1
    for index, value in enumerate(barang):
        if value == j[1]:
            k = index+1
            break
    
    if [j[2],j[3], k] not in pembelian:
        pembelian.append([j[2],j[3], k])

In [14]:
c.execute("DELETE FROM BARANG")
for index, i in enumerate(barang):
    c.execute("INSERT INTO BARANG VALUES(" + str(index+1) + ", '" + str(i) + "')")

c.execute("DELETE FROM PEMBELIAN")
for index, i in enumerate(pembelian):
    c.execute("INSERT INTO PEMBELIAN VALUES(" + str(index+1) + ", '" + str(i[0]) + "','" + str(i[1]) +"'," + str(i[2]) + ")")
    
conn.commit()

In [15]:
c.execute('''SELECT * FROM BARANG''')

BARANG = []
for row in c.fetchall():
    BARANG.append(row)
print("============================BARANG============================")
print(BARANG)
print("==============================================================")

c.execute('''SELECT * FROM PEMBELIAN''')

PEMBELIAN = []
for row in c.fetchall():
    PEMBELIAN.append(row)
print("============================PEMBELIAN============================")
print(PEMBELIAN)
print("=================================================================")

============================BARANG============================
[(1, 'JAKET'), (2, 'KEMEJA FLANEL BENHILL'), (3, 'CELANA CHINOS EMBA'), (4, 'KAOS  3SECOND'), (5, 'CELANA PENDEK NEVADA KNIT BASIC'), (6, 'COLE SLIP ON'), (7, 'KEMEJA NEVADA MEDIUM CHECKS'), (8, 'KEMEJA NEVADA LONG SLEEVE'), (9, 'DASTER NEVADA LENGAN PENDEK HAFLEY CAT'), (10, 'TATUIS MUKENA LAYYANAH KHANSA'), (11, 'ANNISA ROK FRONT DART'), (12, 'NEVADA SANDALS EVAN JEPIT'), (13, 'PAKALOLO FLIP FLOP SANDALS'), (14, 'OVS SNEAKERS SPORT NET'), (15, 'MARVEL SPIDERMAN FLIP FLOPS'), (16, 'KAOS KAKI MARVEL SIDERMAN'), (17, 'TRISET FLAT SANDAL THONG'), (18, 'FILA SEPATU OLAHRAGA LENNON'), (19, 'WATCHOUT SEPATU FORMAL KLASIK'), (20, 'MAYBELLINE LIQUID LIPSTICK'), (21, 'WARDAH EYESHADOW A'), (22, 'OVS HOODIE FULL ZIPPER'), (23, 'PHENOMENAL T-SHIRT CHAIN'), (24, 'CONNEXION CELANA FORMAL ANKLE'), (25, 'NEVADA SWATSHIRT LOVE KNIT'), (26, 'ST. YVES BALLERINAS BUCKLE'), (27, 'NEVADA FLAT BALLERINA'), (28, 'NEVADA SNEAKERS WOMEN'), (29, 'L